In [ ]:
import pandas as pd
from datascience import *
%matplotlib inline

# Internal Developmental Notebook 
#### BUDS Project: Spotify

## Data Cleaning

In [ ]:
spotify = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')
spotify.head()

In [ ]:
spotify['creation_year'] = [x.year for x in spotify['track_album_release_date'].apply(pd.to_datetime)]
spotify.head()

In [ ]:
spotify.to_csv('data/spotify.csv', index = False)

## Loading Data

In [ ]:
spotify = Table.read_table('spotify.csv')
spotify

### Accessing columns
The simplest way to access data in a table is the `column` method, which takes in the name of the column as its only argument. It returns the contents of the column as an array. Imagine that we want to know the duration of all the songs in our dataset. We can access the `duration_ms` column using the column method.

In [ ]:
spotify.column("duration_ms")

We can give this array a name in order to more easily work with the values in it. The name can be whatever we want, but for clarity we'll simply name it `duration`.

In [ ]:
duration = spotify.column("duration_ms")
duration

Since `duration` is just an array, we can perform array operations on it.

### Functions Applied to Track Durations
Right now, the `duration` array contains the length of each song in milliseconds. However, that's not a common measurement when describing the length of a song - often, we use minutes and seconds. Using array arithmetic, we can find the length of each song in seconds and in minutes. There are 1000 milliseconds in a second, and 60 seconds in a minute. First, we will convert milliseconds to seconds.

In [ ]:
duration_seconds = duration / 1000
duration_seconds

Now that we have a new array of the duration of songs in seconds, we can use that to convert to duration in minutes using similar array math.

In [ ]:
duration_minutes = duration_seconds / 60
duration_minutes 

If you are interested in the average length of a song, you can find the mean of the array with built in methods.

In [ ]:
duration_minutes.mean()

We can use the table `with_columns` method to create a copy of the `spotify` table with the duration in minutes as a new column.

In [ ]:
spotify.with_columns('duration_min', duration_minutes)

## Example: Spotify
Our Spotify data contains a lot of numerical information about different aspects of the songs, such as danceability, liveliness, tempo, energy, etc. In this section, we are interested in the categorical information in our dataset, such as the playlist each song comes from or the genre. There are almost 33,000 songs in our dataset, so let's do some investigating. What are the most popular genres? We can figure this out by grouping by the playlist genre.

In [ ]:
spotify.group('playlist_genre')

In our dataset, it looks like the most popular genre is EDM. We can create a barchart to show how the other genres compare.

In [ ]:
genre_counts = spotify.group('playlist_genre')
genre_counts.barh('playlist_genre', 'count')

The bars aren't ordered by length, so it's a little difficult to compare. We can use a more complicated method of sorting to make our bar chart more readable.

In [ ]:
genre_counts_sorted = genre_counts.sort('count', descending=True)
genre_counts_sorted

In [ ]:
genre_counts_sorted.barh('playlist_genre', 'count')

This makes it much easier to see the ranking of most popular genres.

### Top Artists
Let's take a look at all the artists in the dataset. We can take a look at the top 25 artists based on the number of songs they have in our dataset. We'll follow a similar method as we did when grouping by genre above. First, we will group our data by artist and sort by count.

In [ ]:
#Here, we will group and sort in the same line.

artists_grouped = spotify.group('track_artist').sort('count', descending=True)
artists_grouped

artists_grouped = spotify.group('track_artist').sort('count', descending=True)
artists_grouped

The output displays the top 10 artists, but there are over 10,000 artists in our dataset. A visualization with every artist would be very hard to read, so we will focus on the top 25 artists.

In [ ]:
top_artists = artists_grouped.take[:25]
top_artists

Creating a bar chart of the number of songs for each artists lets us easily compare the differences between artists.

In [ ]:
top_artists.barh('track_artist', 'count')

A quick look through the top 25 shows us that all of the top artists are males.

### Analyzing Playlist Popularity
In our dataset, each song is listed as belonging to a particular playlist, and each song is given a "popularity score", called the `track_popularity`. Using the `track_popularity`, we can calculate an "aggregate popularity" for each playlist, which is just the sum of all the popularity scores for the songs on the playlist.

In order to create this aggregate popularity score, we need to group our data by playlist, and sum all of the popularity scores. First, we will create a subset of our `spotify` table using the `select` method. This lets us create a table with only the relevant columns we want. In this case, we only care about the name of the playlist and the popularity of each track. Keep in mind that each row still represents one track, even though we no longer have the track title in our table.

In [ ]:
spotify_subset = spotify.select(['playlist_name', 'track_popularity'])
spotify_subset

Using the `group` method, we can get a count of the number of songs in each playlist. 

In [ ]:
playlists = spotify_subset.group('playlist_name')
playlists

We can use the `group` method again, this time passing in a second argument `collect`, which says that we want to take the sum rather than the count when grouping. This results in a table with the total aggregate popularity of each playlist.

In [ ]:
total_playlist_popularity = spotify_subset.group('playlist_name', collect = sum)
total_playlist_popularity

We can once again use the `column` method to access just the `track_popularity sum` column, and add it to our playlists table using the `with_column` method.

In [ ]:
agg_popularity = total_playlist_popularity.column('track_popularity sum')
playlists = playlists.with_column('aggregate_popularity', agg_popularity)
playlists

Do you think that the most popular playlist would be the one with the highest `aggregate_popularity` score, or the one with the highest number of songs? We can sort our `playlists` table and compare the outputs.

In [ ]:
playlists.sort('count', descending=True)

In [ ]:
playlists.sort('aggregate_popularity', descending=True)

Comparing these two outputs shows us that the "most popular playlist" depends on how we judge popularity. If we have a playlist that has only a few songs, but each of those songs are really popular, should that playlist be higher on the popularity rankings? By way of calculation, playlists with more songs will have a higher aggregate popularity, since more popularity values are being added together. We want a metric that will let us judge the actual quality and popularity of a playlist, not just how many songs it has.

In order to take into account the number of songs on each playlist, we can calculate the "average popularity" of each song on the playlist, or the proportion of aggregate popularity that each song takes up. We can do this by dividing `aggregate_popularity` by `count`. Remember, since the columns are just arrays, we can use array math to calculate these values.

In [ ]:
avg_popularity = playlists.column('aggregate_popularity') / playlists.column('count')

We can then add this average popularity back into our `playlists` table.

In [ ]:
playlists = playlists.with_column('average_popularity', avg_popularity)
playlists

Let's see if our "most popular playlist" changes when we judge popularity by the average popularity of the songs on a playlist.

In [ ]:
playlists.sort('average_popularity', descending=True)

Looking at the table above, we notice that 8/10 of the top 10 most popular playlists by the `average_popularity` metric are playlists with less than 100 songs. Just because a playlist has a lot of songs, or a high aggregate popularity, doesn't mean that the average popularity of a song on that playlist is high. Our new metric of `average_popularity` lets us rank playlists where the size of a playlist has no effect on it's overall score. We can visualize the top 25 playlists by average popularity in a bar chart.

In [ ]:
top_25_playlists = playlists.sort('average_popularity', descending=True).take[:25]
top_25_playlists.barh('playlist_name', 'average_popularity')

Creating a new metric like `average_popularity` helps us more accurately and fairly measure the popularity of a playlist. 

We saw before when looking at the top 25 artists that they were all male. Now looking at the top playlists, we see that the current landscape of popular playlists and music may have an effect on the artists that are popular. For example, the RapCaviar is the second most popular playlist, and generally there tends to be fewer female rap artists than male. This shows that the current landscape of popular music can affect the types of artists topping the charts.

## Towards numerical variables
Most of the time, we are not solely interested in categorical variables. Our Spotify data also contains information on the release date of the track, so we can see how many songs were released each year.

In [ ]:
grouped_years = spotify.select('track_artist', 'creation_year').group('creation_year').sort('count', descending=True)
grouped_years

Sorting by the descending count lets us see which years had the most songs.

In [ ]:
grouped_years.barh('creation_year', 'count')

The longest bars correspond to most recent years, but if you look closely, the years are not listed in chronological order. Years are a fixed chronological unit that have a specified order, but the bar chart is treating year as a categorical variable. We can sort by year instead of by count in order to get a chronological bar chart.

In [ ]:
grouped_years.sort('creation_year', descending=True).barh('creation_year', 'count')

Now we can see that more songs have been released in more recent years. The bar for 2020 is much shorter than the bar for 2019, because the data was downloaded in January 2020, when there weren't very many new songs out yet.